In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
!pip install --quiet timm==0.9.2 albumentations==1.4.3 scikit-image scikit-learn
!pip install --quiet git+https://github.com/facebookresearch/segment-anything.git

import os, time, random
from pathlib import Path
from glob import glob
from typing import List

import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)


  Preparing metadata (setup.py) ... done


In [11]:
CONFIG = {
    'DATA_DIR': '/kaggle/input/brain-tumor-for-segmentation/Brain tumor segmentation',  # your dataset
    'IMG_EXTS': ('.png','.jpg','.jpeg'),
    'IMG_SIZE': 224,
    'NUM_CLASSES': 2,
    'BATCH_SIZE': 8,
    'EPOCHS': 10,
    'LR': 1e-4,
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu',
    'CHECKPOINT_DIR': '/kaggle/working/checkpoints',
    'GENERATED_MASKS_DIR': '/kaggle/working/masks_auto',
    'SAM_CHECKPOINT': '/kaggle/input/sam-vit/sam_vit_b_01ec64.pth',  # set this path
    'NUM_WORKERS': 0,
}
os.makedirs(CONFIG['CHECKPOINT_DIR'], exist_ok=True)
os.makedirs(CONFIG['GENERATED_MASKS_DIR'], exist_ok=True)

print("Device:", CONFIG['DEVICE'])
print("Images root:", CONFIG['DATA_DIR'])
print("Masks will be saved to:", CONFIG['GENERATED_MASKS_DIR'])


Device: cuda
Images root: /kaggle/input/brain-tumor-for-segmentation/Brain tumor segmentation
Masks will be saved to: /kaggle/working/masks_auto


In [12]:
DEVICE = CONFIG['DEVICE']

sam = sam_model_registry["vit_b"](checkpoint=CONFIG['SAM_CHECKPOINT'])
sam.to(DEVICE)

mask_generator = SamAutomaticMaskGenerator(
    sam,
    points_per_side=32,
    pred_iou_thresh=0.88,
    stability_score_thresh=0.92
)


In [ ]:
def collect_images(root_dir: str, exts=('.png','.jpg','.jpeg')) -> List[str]:
    files = []
    p = Path(root_dir)
    for ext in exts:
        files += [str(x) for x in p.rglob(f'*{ext}')]
    return sorted(files)

def read_image(path: str, size:int=224):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (size, size))
    return img

def generate_sam_mask(img_path: str):
    img = read_image(img_path, CONFIG['IMG_SIZE'])
    masks = mask_generator.generate(img)
    # combine all masks into one binary mask
    final_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    for m in masks:
        final_mask = np.maximum(final_mask, m['segmentation'].astype(np.uint8))
    final_mask = cv2.resize(final_mask, (CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']), interpolation=cv2.INTER_NEAREST)
    return final_mask

# Generate and save pseudo-masks
images = collect_images(CONFIG['DATA_DIR'], CONFIG['IMG_EXTS'])
for i, p in enumerate(images):
    fname = os.path.basename(p)
    out_mask = os.path.join(CONFIG['GENERATED_MASKS_DIR'], fname)
    if os.path.exists(out_mask): continue
    mask = generate_sam_mask(p)
    cv2.imwrite(out_mask, (mask*255).astype('uint8'))
print("SAM pseudo-mask generation done")


In [ ]:
class BrainSegDataset(Dataset):
    def __init__(self, image_paths: List[str], mask_dir: str, img_size:int=224, transform=None):
        self.image_paths = image_paths
        self.mask_dir = mask_dir
        self.img_size = img_size
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        p = self.image_paths[idx]
        fname = os.path.basename(p)
        img = read_image(p, self.img_size)
        mask = cv2.imread(os.path.join(self.mask_dir, fname), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            mask = np.zeros((self.img_size, self.img_size), dtype=np.uint8)
        else:
            mask = cv2.resize(mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
            mask = (mask>127).astype(np.uint8)
        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            return augmented['image'], augmented['mask']
        else:
            img_t = torch.from_numpy(img.transpose(2,0,1)).float()/255.0
            mask_t = torch.from_numpy(mask).long()
            return img_t, mask_t


In [ ]:
train_tf = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(),
    ToTensorV2()
])
val_tf = A.Compose([A.Normalize(), ToTensorV2()])


In [ ]:
n = len(images)
idxs = np.arange(n); np.random.seed(SEED); np.random.shuffle(idxs)
split = int(0.8*n)
train_imgs = [images[i] for i in idxs[:split]]
val_imgs   = [images[i] for i in idxs[split:]]

train_ds = BrainSegDataset(train_imgs, CONFIG['GENERATED_MASKS_DIR'], img_size=CONFIG['IMG_SIZE'], transform=train_tf)
val_ds   = BrainSegDataset(val_imgs, CONFIG['GENERATED_MASKS_DIR'], img_size=CONFIG['IMG_SIZE'], transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=CONFIG['NUM_WORKERS'], pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=CONFIG['NUM_WORKERS'], pin_memory=True)


In [ ]:
class SwinUNet(nn.Module):
    def __init__(self, num_classes=2, img_size=224):
        super().__init__()
        self.encoder = timm.create_model('swin_base_patch4_window7_224', pretrained=True, features_only=True)
        self.encoder_out_channels = self.encoder.feature_info.channels()
        # Decoder
        self.up4 = nn.ConvTranspose2d(self.encoder_out_channels[-1], 512, kernel_size=2, stride=2)
        self.conv4 = nn.Conv2d(512+512, 512, 3, padding=1)
        self.up3 = nn.ConvTranspose2d(512, 256, 2,2)
        self.conv3 = nn.Conv2d(256+256, 256, 3,1,1)
        self.up2 = nn.ConvTranspose2d(256,128,2,2)
        self.conv2 = nn.Conv2d(128+128,128,3,1,1)
        self.up1 = nn.ConvTranspose2d(128,64,2,2)
        self.conv1 = nn.Conv2d(64+64,64,3,1,1)
        self.head = nn.Conv2d(64,num_classes,1)

    def forward(self,x):
        feats = self.encoder(x)
        e1,e2,e3,e4 = feats
        d4 = self.up4(e4)
        d4 = torch.cat([d4,e3],dim=1)
        d4 = self.conv4(d4)
        d3 = self.up3(d4)
        d3 = torch.cat([d3,e2],dim=1)
        d3 = self.conv3(d3)
        d2 = self.up2(d3)
        d2 = torch.cat([d2,e1],dim=1)
        d2 = self.conv2(d2)
        d1 = self.up1(d2)
        d1 = self.conv1(d1)
        out = self.head(d1)
        return out


In [ ]:
def dice_loss_logits(logits, target, eps=1e-6):
    prob = torch.sigmoid(logits)
    target = target.unsqueeze(1).float()
    inter = (prob * target).sum(dim=(2,3))
    denom = prob.sum(dim=(2,3)) + target.sum(dim=(2,3))
    dice = (2*inter + eps)/(denom+eps)
    return 1 - dice.mean()

def iou_score(logits, target):
    preds = (torch.sigmoid(logits).squeeze(1) > 0.5).long()
    inter = (preds & target).sum().float()
    union = (preds | target).sum().float()
    return (inter/union) if union>0 else torch.tensor(1.0)


In [ ]:
device = CONFIG['DEVICE']
model = SwinUNet(num_classes=CONFIG['NUM_CLASSES'], img_size=CONFIG['IMG_SIZE']).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['LR'])
scaler = torch.cuda.amp.GradScaler() if device.startswith('cuda') else None


In [ ]:
def train_one_epoch():
    model.train(); total_loss=0
    for imgs, masks in train_loader:
        imgs = imgs.to(device); masks = masks.to(device)
        optimizer.zero_grad()
        if scaler:
            with torch.cuda.amp.autocast():
                logits = model(imgs)
                bce = F.binary_cross_entropy_with_logits(logits.squeeze(1), masks.float())
                loss = bce + dice_loss_logits(logits, masks)
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            logits = model(imgs)
            loss = F.binary_cross_entropy_with_logits(logits.squeeze(1), masks.float()) + dice_loss_logits(logits, masks)
            loss.backward(); optimizer.step()
        total_loss += loss.item()*imgs.size(0)
    return total_loss / len(train_loader.dataset)

def validate_epoch():
    model.eval(); total_loss=0; total_iou=0
    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs = imgs.to(device); masks = masks.to(device)
            logits = model(imgs)
            loss = F.binary_cross_entropy_with_logits(logits.squeeze(1), masks.float()) + dice_loss_logits(logits, masks)
            total_loss += loss.item()*imgs.size(0)
            total_iou += iou_score(logits, masks).item()*imgs.size(0)
    return total_loss/len(val_loader.dataset), total_iou/len(val_loader.dataset)


In [ ]:
best_iou=0.0
for epoch in range(1, CONFIG['EPOCHS']+1):
    t0 = time.time()
    train_loss = train_one_epoch()
    val_loss, val_iou = validate_epoch()
    t1 = time.time()
    print(f"Epoch {epoch}/{CONFIG['EPOCHS']} | {t1-t0:.1f}s | train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | val_iou {val_iou:.4f}")
    ckpt = {'epoch':epoch,'model_state':model.state_dict(),'optim_state':optimizer.state_dict(),'val_iou':val_iou}
    torch.save(ckpt, os.path.join(CONFIG['CHECKPOINT_DIR'], f'epoch_{epoch}.pth'))
    if val_iou>best_iou:
        best_iou=val_iou
        torch.save(ckpt, os.path.join(CONFIG['CHECKPOINT_DIR'],'best.pth'))
print("Training finished. Best val IoU:", best_iou)


In [ ]:
def predict_mask(model, img_path):
    model.eval()
    img = read_image(img_path, CONFIG['IMG_SIZE'])
    inp = torch.from_numpy(img.transpose(2,0,1)).float().unsqueeze(0).to(device)/255.0
    with torch.no_grad():
        logits = model(inp)
        pred = (torch.sigmoid(logits).squeeze(1).cpu().numpy()[0]>0.5).astype('uint8')
    return img, pred

# visualize
for i in range(min(6,len(val_imgs))):
    p = val_imgs[i]
    img, pred = predict_mask(model, p)
    gt = cv2.imread(os.path.join(CONFIG['GENERATED_MASKS_DIR'], os.path.basename(p)), cv2.IMREAD_GRAYSCALE)
    if gt is None: gt=np.zeros_like(pred)
    else: gt=(gt>127).astype('uint8')
    plt.figure(figsize=(10,3))
    plt.subplot(1,3,1); plt.imshow(img); plt.title('Image'); plt.axis('off')
    plt.subplot(1,3,2); plt.imshow(pred); plt.title('Predicted Mask'); plt.axis('off')
    plt.subplot(1,3,3); plt.imshow(gt); plt.title('SAM Pseudo Mask'); plt.axis('off')
    plt.show()
